In [1]:
import os
import glob
import xarray as xr

In [2]:
import numpy as np

In [3]:
np.__version__

'1.22.1'

In [4]:
file = os.listdir('1983')
file

['PERSIANN-CDR_v01r01_19830101_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830102_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830103_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830104_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830105_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830106_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830107_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830108_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830109_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830110_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830111_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830112_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830113_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830114_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830115_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830116_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830117_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830118_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830119_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830120_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830121_c20140523.nc',
 'PERSIANN-CDR_v01r01_19830122_c20

In [5]:
folder_path = os.getcwd() 
folder_path

'c:\\Users\\Shubham\\Documents\\TANISHQ\\Project\\1983\\'

In [6]:
# for i in file:
file_path = os.path.join(folder_path, '*.nc')
files = sorted(glob.glob(file_path))

files

['c:\\Users\\Shubham\\Documents\\TANISHQ\\Project\\1983\\PERSIANN-CDR_v01r01_19830101_c20140523.nc',
 'c:\\Users\\Shubham\\Documents\\TANISHQ\\Project\\1983\\PERSIANN-CDR_v01r01_19830102_c20140523.nc',
 'c:\\Users\\Shubham\\Documents\\TANISHQ\\Project\\1983\\PERSIANN-CDR_v01r01_19830103_c20140523.nc',
 'c:\\Users\\Shubham\\Documents\\TANISHQ\\Project\\1983\\PERSIANN-CDR_v01r01_19830104_c20140523.nc',
 'c:\\Users\\Shubham\\Documents\\TANISHQ\\Project\\1983\\PERSIANN-CDR_v01r01_19830105_c20140523.nc',
 'c:\\Users\\Shubham\\Documents\\TANISHQ\\Project\\1983\\PERSIANN-CDR_v01r01_19830106_c20140523.nc',
 'c:\\Users\\Shubham\\Documents\\TANISHQ\\Project\\1983\\PERSIANN-CDR_v01r01_19830107_c20140523.nc',
 'c:\\Users\\Shubham\\Documents\\TANISHQ\\Project\\1983\\PERSIANN-CDR_v01r01_19830108_c20140523.nc',
 'c:\\Users\\Shubham\\Documents\\TANISHQ\\Project\\1983\\PERSIANN-CDR_v01r01_19830109_c20140523.nc',
 'c:\\Users\\Shubham\\Documents\\TANISHQ\\Project\\1983\\PERSIANN-CDR_v01r01_19830110_c2014

In [ ]:
def merge_netCDF_files(input_folder, output_file):
    # Create a list of all NetCDF files sorted by their filename (assuming filenames contain dates)
    file_pattern = os.path.join(input_folder, '*.nc4')
    files = sorted(glob.glob(file_pattern))
    
    # Load datasets into a list and concatenate them manually
    datasets = [xr.open_dataset(file) for file in files]
    combined_ds = xr.concat(datasets, dim='time')

    # Save the combined dataset to a new NetCDF file
    combined_ds.to_netcdf(output_file)
    for ds in datasets:  # Close all datasets to free up resources
        ds.close()

# Usage example:
input_folder = 'C:/Users/Shubham/Desktop/TRMM'
output_file = 'C:/Users/Shubham/Desktop/TRMM/TRMMmerged_data.nc4'
merge_netCDF_files(input_folder, output_file)

In [3]:
import xarray as xr
import os
import glob
import numpy as np
import warnings
from xarray.coding.variables import SerializationWarning

def preprocess_crop_and_merge(input_folder, output_file):
    file_pattern = os.path.join(input_folder, '*.nc')
    files = sorted(glob.glob(file_pattern))
    
    datasets = []
    for file in files:
        with warnings.catch_warnings():
            # Ignore warnings specific to each dataset load
            warnings.simplefilter("ignore")
            ds = xr.open_dataset(file)
        
        # Clean up fill values and missing values
        if 'precipitation' in ds.data_vars:
            # Ensure that fill values are consistent
            ds['precipitation'].encoding.pop('missing_value', None)
            ds['precipitation'].encoding['_FillValue'] = -9999.0
        
        # Get the indices using np.where and crop the data
        lat_indices = np.where((ds.lat >= 25) & (ds.lat <= 30))[0]
        lon_indices = np.where((ds.lon >= 83) & (ds.lon <= 90))[0]
        ds_cropped = ds.isel(lat=lat_indices, lon=lon_indices)
        
        datasets.append(ds_cropped)
    
    # Concatenate all datasets along the time dimension
    combined_ds = xr.concat(datasets, dim='time', data_vars='minimal', coords='minimal', compat='override')

    # Handle any potential serialization warnings due to encoding settings when saving
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=SerializationWarning)
        combined_ds.to_netcdf(output_file)
    
    # Close datasets to free up resources
    for ds in datasets:
        ds.close()

# Example usage:
input_folder = r'C:\Users\Ankit\Documents\Vedanshi\nc_merged\merged_noaa'
output_file = r'C:\Users\Ankit\Documents\Vedanshi\nc_merged\total_merged_data.nc'
preprocess_crop_and_merge(input_folder, output_file)

In [19]:
xr.__version__

'2023.12.0'

In [6]:
merged_ds = xr.open_dataset(r'C:\Users\Ankit\Documents\Vedanshi\nc_merged\total_merged_data.nc')

In [9]:
merged_ds

<xarray.Dataset> Size: 34MB
Dimensions:        (lon: 28, nv: 2, lat: 20, time: 14975)
Coordinates:
  * time           (time) datetime64[ns] 120kB 1983-01-01 ... 2023-12-31
  * lat            (lat) float32 80B 29.88 29.62 29.38 ... 25.62 25.38 25.12
  * lon            (lon) float32 112B 83.12 83.38 83.62 ... 89.38 89.62 89.88
Dimensions without coordinates: nv
Data variables:
    lon_bnds       (lon, nv) float32 224B ...
    lat_bnds       (lat, nv) float32 160B ...
    precipitation  (time, lon, lat) float32 34MB ...
Attributes: (12/37)
    datetime:                   1983-01-01
    Conventions:                CF-1.6
    title:                      NOAA Climate Data Record (CDR) of Precipitati...
    comment:                    Daily PERSIANN-CDR precipitation estimates fo...
    source:                     GRIDSAT-B1.1983.01.01.00.v02r01.nc,GRIDSAT-B1...
    Metadata_Conventions:       CF-1.6,Unidata Dataset Discovery v1.0, NOAA C...
    ...                         ...
    contributor_role:           PrincipalInvestigator, Professor, Author, Author
    cdr_program:                NOAA Climate Data Record Program
    cdr_variable:               precipitation
    metadata_link:              gov.noaa.ncdc:C00854
    product_version:            v01r01
    spatial_resolution:         0.25 degrees